In [ ]:
import os
import re
import json
import joblib
import numpy as np
import pandas as pd
import textwrap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Dropout, Concatenate, Bidirectional
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam
from nltk.tokenize import word_tokenize

# ------------------------------#
# Part 1: Data Loading & Model Prediction
# ------------------------------#

# Set file paths for saved resources
output_dir = 'E:/Concrete_Freeze_Thaw_Damage_Prediction_Results/GRU_outputs'
vocab_file = os.path.join(output_dir, 'vocab_index.json')
max_lengths_file = os.path.join(output_dir, 'max_lengths.csv')
scaler_file = os.path.join(output_dir, 'scaler.pkl')
model_weights_file = os.path.join(output_dir, 'model_fold_4.h5')

# Load vocabulary index
with open(vocab_file, 'r', encoding='utf-8') as f:
    word_index = json.load(f)

# Load maximum lengths (one per text input)
max_lengths_df = pd.read_csv(max_lengths_file)
max_lengths = max_lengths_df['Max_Length'].tolist()

# Load the scaler for numeric features
scaler = joblib.load(scaler_file)

# Define vocabulary size and embedding dimension (consistent with training)
vocab_size = 2025
embedding_dim = 64

# Define new sample texts and numeric data (replace with actual sample data as needed)
new_sample_text = [
    '5% sodium sulfate solution', 
    'In accordance with ASTM C666, a KDR-V9 freeze-thaw box was used for the freeze-thaw tests. There are 28 test kits in a box and the test specimens were all placed in the kit. Distilled water was added to the test kits that were sitting 20 mm higher than the concrete specimen. The samples were soaked in the test box for 4 days before adding the etching solution and the water to a level sitting 10 mm above the test specimens. The solution was replaced regularly to ensure solution constant concentration. A temperature sensor was placed in a cavity of a diameter of 12 mm in the center of the specimen to maintain the specimen at (17 ± 2) °C and (8 ± 2) °C. Each freeze-thaw cycle was completed within 2–4 h. The thawing time was never less than 1/4 of the duration of a freeze-thaw cycle.',
    'The damage of the Natural Concrete specimens after 100 and 200 freeze-thaw cycles was not significant. After 300 freeze-thaw cycles, small pits appeared on the surface but the aggregates were not exposed. In the Concrete with 30% replacement of recycled aggregates group, small pits appeared after 100 freeze-thaw cycles. After 200 cycles, the surface was sanded and the mortar was peeled off. After 300 cycles, the surface partially exposed the aggregate. The specimens of the Concrete with 50% replacement of recycled aggregates group showed small pits on the surface after 100 freeze-thaw cycles, but the peeling was not serious. When the number of freeze-thaw cycles increased, the damage gradually expanded from the outside to the inside. After 200 cycles, the aggregate was gradually exposed. After 300 freeze-thaw cycles, all aggregates were exposed. The surface of the specimens from the Concrete with 100% replacement of recycled aggregates group peeled off very quickly. The sand first came from the surface and was then peeled off.',
    'For freeze-thaw cycles in the presence of sulfate, the relative dynamic modulus of elasticity of the recycled aggregate concrete goes through three stages: a steady drop, an accelerated drop, and a rapid decrease. For the first freeze-thaw cycles, the sulfate attack focused on the concrete surface, slowly counter-acting the decrease in the relative dynamic modulus of elasticity. When the freeze-thaw erosion intensifies, the sulfate reacts with the cementitious material to form an erosion product, namely ettringite, which causes damage to the recycled aggregate concrete and accelerates the decrease. With the progress of freeze-thaw failure and the increased formation of erosion products, cracks spread and, in conjunction with freeze-thaw erosion, the damage to the recycled aggregate concrete is accelerated. Moreover, because the recycled aggregate itself cracks when broken, freeze-thaw erosion significantly accelerates when it enters the recycled aggregate concrete.'
]

new_sample_numeric = [0.45, 0.57, 355, 0, 0, 0, 160, 585, 0, 1300, 26.8, 3.1, 0, 0, 0, 0, 100, 1]  # Replace with actual numerical data if needed

# Text preprocessing function: convert to lowercase and remove certain punctuation
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[,.:]', '', text)
    return text

# Tokenize texts and create padded sequences
tokenized_texts = []
padded_sequences = []
for i, text in enumerate(new_sample_text):
    processed_text = preprocess_text(text)
    tokens = word_tokenize(processed_text)
    tokenized_texts.append(tokens)
    sequence = [word_index.get(word, 0) for word in tokens]
    padded_sequence = pad_sequences([sequence], maxlen=max_lengths[i], padding='post', truncating='post')[0]
    padded_sequences.append(padded_sequence)

# Scale numeric data
new_sample_numeric_scaled = scaler.transform(np.array([new_sample_numeric], dtype=np.float32))

# Define and create a bidirectional GRU model
def create_model(max_lengths, vocab_size, embedding_dim):
    inputs = []
    final_states = []
    sequence_outputs = []

    for i, max_len in enumerate(max_lengths):
        # Input layer for text sequence
        input_layer = Input(shape=(max_len,), dtype='int32', name=f"text_input_{i+1}")
        embedding_layer = Embedding(
            input_dim=vocab_size + 1,
            output_dim=embedding_dim,
            input_length=max_len,
            trainable=True,
            mask_zero=True,
            name=f"embedding_{i+1}"
        )(input_layer)

        # Bidirectional GRU with return_sequences and return_state enabled
        gru_layer = Bidirectional(
            GRU(16, return_sequences=True, return_state=True),
            name=f"bigru_{i+1}"
        )
        # For Bidirectional GRU, outputs: [sequence_output, forward_h, backward_h]
        gru_outputs = gru_layer(embedding_layer)
        sequence_output = gru_outputs[0]  # Shape: (batch_size, sequence_length, units*2)
        forward_h = gru_outputs[1]        # Shape: (batch_size, units)
        backward_h = gru_outputs[2]       # Shape: (batch_size, units)

        # Concatenate final hidden states from both directions
        final_state = Concatenate(name=f"final_state_{i+1}")([forward_h, backward_h])

        inputs.append(input_layer)
        final_states.append(final_state)
        sequence_outputs.append(sequence_output)

    # Numeric input
    input_numeric = Input(shape=(18,), dtype='float32', name="numerical_input")
    inputs.append(input_numeric)

    # Combine final states and numeric features
    combined_features = Concatenate(name="combined_features")(final_states + [input_numeric])

    # Fully connected layers for prediction
    x = Dense(128, activation='relu', kernel_initializer=HeNormal(), name="dense_1")(combined_features)
    x = Dropout(0.2, name="dropout_1")(x)
    x = Dense(64, activation='relu', kernel_initializer=HeNormal(), name="dense_2")(x)
    x = Dropout(0.2, name="dropout_2")(x)
    x = Dense(32, activation='relu', kernel_initializer=HeNormal(), name="dense_3")(x)
    x = Dropout(0.2, name="dropout_3")(x)
    final_output = Dense(2, name="final_output")(x)

    # Model outputs: primary prediction and the sequence outputs for analysis
    model = Model(inputs=inputs, outputs=[final_output] + sequence_outputs, name="analysis_model")

    # Compile the model. Only compute loss on the primary output.
    loss_dict = {'final_output': 'mae'}
    loss_weights = {'final_output': 1.0}
    for i in range(len(sequence_outputs)):
        loss_dict[f"bigru_{i+1}"] = None
        loss_weights[f"bigru_{i+1}"] = 0.0

    model.compile(optimizer=Adam(learning_rate=0.001), loss=loss_dict, loss_weights=loss_weights)
    return model

# Build the model and load weights
model = create_model(max_lengths, vocab_size, embedding_dim)
model.load_weights(model_weights_file)

# Perform prediction using the new sample inputs
text_inputs = [seq.reshape(1, -1) for seq in padded_sequences]  # Reshape text inputs
all_inputs = text_inputs + [new_sample_numeric_scaled]         # Append numeric input
model_outputs = model.predict(all_inputs)

# Extract outputs: final prediction and hidden sequence outputs from each text input
final_prediction = model_outputs[0]
sequence_outputs = model_outputs[1:]
print("Final Prediction Output:", final_prediction)
for i, seq_output in enumerate(sequence_outputs):
    print(f"Sequence output {i + 1} shape:", seq_output.shape)

# ------------------------------#
# Part 2: Hidden State Distance Computation and Excel Export
# ------------------------------#
# For each text input's sequence output, compute distances on the forward and backward hidden states,
# standardize and normalize them, then save the results to an Excel file (separate sheets for forward and backward)

for i, seq_output in enumerate(sequence_outputs):
    # Remove batch dimension (assumes batch size = 1)
    seq_output = seq_output[0]
    tokens = tokenized_texts[i]
    actual_length = len(tokens)
    seq_output = seq_output[:actual_length]  # Remove padded part

    # Split into forward and backward hidden states (assuming GRU units=16 each direction)
    forward_hidden = seq_output[:, :16]
    backward_hidden = seq_output[:, 16:]

    # Initialize distance arrays
    forward_manhattan = [0] * len(tokens)
    forward_cosine = [0] * len(tokens)
    backward_manhattan = [0] * len(tokens)
    backward_cosine = [0] * len(tokens)

    # Compute forward distances
    for j in range(1, len(forward_hidden)):
        manhattan_dist = np.sum(np.abs(forward_hidden[j] - forward_hidden[j-1]))
        forward_manhattan[j] = manhattan_dist

        dot_prod = np.dot(forward_hidden[j-1], forward_hidden[j])
        norm_prod = np.linalg.norm(forward_hidden[j-1]) * np.linalg.norm(forward_hidden[j])
        cosine_dist = 1 - (dot_prod / norm_prod) if norm_prod != 0 else 0
        forward_cosine[j] = cosine_dist

    # Compute backward distances
    for j in range(1, len(backward_hidden)):
        manhattan_dist = np.sum(np.abs(backward_hidden[j] - backward_hidden[j-1]))
        backward_manhattan[j] = manhattan_dist

        dot_prod = np.dot(backward_hidden[j], backward_hidden[j-1])
        norm_prod = np.linalg.norm(backward_hidden[j]) * np.linalg.norm(backward_hidden[j-1])
        cosine_dist = 1 - (dot_prod / norm_prod) if norm_prod != 0 else 0
        backward_cosine[j] = cosine_dist

    # Reverse backward distances for alignment with forward order
    backward_manhattan_reversed = backward_manhattan[::-1]
    backward_cosine_reversed = backward_cosine[::-1]

    # Create DataFrames for forward and backward distances
    df_forward = pd.DataFrame({
        'Token (Forward)': tokens,
        'Manhattan Distance (Forward)': forward_manhattan,
        'Cosine Distance (Forward)': forward_cosine
    })

    df_backward = pd.DataFrame({
        'Token (Backward)': tokens,
        'Manhattan Distance (Backward)': backward_manhattan_reversed,
        'Cosine Distance (Backward)': backward_cosine_reversed
    })

    # Standardize and normalize (forward)
    if len(df_forward) > 1:
        scaler_std = StandardScaler()
        forward_norm_cols = ['Manhattan Distance (Forward)', 'Cosine Distance (Forward)']
        standardized_forward = scaler_std.fit_transform(df_forward[forward_norm_cols].iloc[1:])
        df_forward.loc[1:, [f'{col} (Standardized)' for col in forward_norm_cols]] = standardized_forward

        minmax_scaler = MinMaxScaler()
        normalized_forward = minmax_scaler.fit_transform(df_forward[[f'{col} (Standardized)' for col in forward_norm_cols]].iloc[1:])
        df_forward.loc[1:, [f'{col} (Normalized)' for col in forward_norm_cols]] = normalized_forward

    # Standardize and normalize (backward)
    if len(df_backward) > 1:
        scaler_std = StandardScaler()
        backward_norm_cols = ['Manhattan Distance (Backward)', 'Cosine Distance (Backward)']
        standardized_backward = scaler_std.fit_transform(df_backward[backward_norm_cols].iloc[:-1])
        df_backward.loc[:len(df_backward)-2, [f'{col} (Standardized)' for col in backward_norm_cols]] = standardized_backward

        minmax_scaler = MinMaxScaler()
        normalized_backward = minmax_scaler.fit_transform(df_backward[[f'{col} (Standardized)' for col in backward_norm_cols]].iloc[:-1])
        df_backward.loc[:len(df_backward)-2, [f'{col} (Normalized)' for col in backward_norm_cols]] = normalized_backward

    # Calculate the normalized distance sums if columns exist
    if 'Manhattan Distance (Forward) (Normalized)' in df_forward.columns and 'Cosine Distance (Forward) (Normalized)' in df_forward.columns:
        df_forward['Distance Sum (Forward) (Normalized)'] = df_forward['Manhattan Distance (Forward) (Normalized)'] + df_forward['Cosine Distance (Forward) (Normalized)']
    if 'Manhattan Distance (Backward) (Normalized)' in df_backward.columns and 'Cosine Distance (Backward) (Normalized)' in df_backward.columns:
        df_backward['Distance Sum (Backward) (Normalized)'] = df_backward['Manhattan Distance (Backward) (Normalized)'] + df_backward['Cosine Distance (Backward) (Normalized)']

    # Save the results to an Excel file with separate sheets for forward and backward
    output_file = os.path.join(output_dir, f'hidden_states_text_input_{i+1}.xlsx')
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        df_forward.to_excel(writer, sheet_name='Forward', index=False)
        df_backward.to_excel(writer, sheet_name='Backward', index=False)

# ------------------------------#
# Part 3: Visualization from Excel Files
# ------------------------------#
# This section reads the Excel files generated above and visualizes the token mapping with colors
# based on the normalized scores.

def visualized_text(df, score_column, tokens, normalized_scores, cmap, wrapper, excel_file, sheet_name):
    wrapped_text = wrapper.wrap(" ".join(tokens))
    fig, ax = plt.subplots(figsize=(12, 5))
    y_pos = -0.02
    line_height = 0.4
    token_idx = 0

    for line in wrapped_text:
        words = line.split()
        x_pos = 3
        for word in words:
            if token_idx < len(tokens):
                score = normalized_scores.iloc[token_idx]
            else:
                score = 0
            ax.text(x_pos, y_pos, word, fontsize=12, fontweight='bold', family='Times New Roman',
                    color=cmap(score), ha='left', va='center')
            x_pos += len(word) + 1
            token_idx += 1
        y_pos -= line_height

    ax.axis([0, 100, y_pos, 0])
    ax.axis('off')

    # Set directory for saving the visualization
    save_dir = os.path.join(output_dir, "visualization")
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    # Create file name based on Excel file name, score column, and sheet name
    base_name = os.path.basename(excel_file).replace('.xlsx', '')
    save_path = os.path.join(save_dir, f"visualized_text_output_{base_name}_{score_column.replace(' ', '_')}_{sheet_name}.svg")
    print(f"Saving image to: {save_path}")
    plt.savefig(save_path, format='svg', dpi=300)
    plt.show()

def visualized_text_from_excel(excel_files, cmap, wrapper):
    for excel_file in excel_files:
        df_forward = pd.read_excel(excel_file, sheet_name='Forward')
        df_backward = pd.read_excel(excel_file, sheet_name='Backward')
        
        print(f"Processing file: {excel_file}")
        
        tokens_forward = df_forward.iloc[:, 0]  # First column: tokens for forward
        tokens_backward = df_backward.iloc[:, 0]  # First column: tokens for backward
        
        # Use the last column of each sheet (with normalized values; fill NaN with 0)
        forward_scores = df_forward.iloc[:, -1].fillna(0)
        backward_scores = df_backward.iloc[:, -1].fillna(0)
        
        print("Visualizing forward tokens and scores.")
        visualized_text(df_forward, forward_scores.name, tokens_forward, forward_scores, cmap, wrapper, excel_file, 'Forward')
        
        print("Visualizing backward tokens and scores.")
        visualized_text(df_backward, backward_scores.name, tokens_backward, backward_scores, cmap, wrapper, excel_file, 'Backward')

# List all the Excel files generated earlier (adjust paths if necessary)
excel_files = [
    os.path.join(output_dir, "hidden_states_text_input_1.xlsx"),
    os.path.join(output_dir, "hidden_states_text_input_2.xlsx"),
    os.path.join(output_dir, "hidden_states_text_input_3.xlsx"),
    os.path.join(output_dir, "hidden_states_text_input_4.xlsx")
]

# Define colormap and text wrapper settings for visualization
cmap = plt.get_cmap('Blues')
wrapper = textwrap.TextWrapper(width=90)

# Call the visualization function
visualized_text_from_excel(excel_files, cmap, wrapper)
